# 从亚马逊云计算上检索或读取公开科学数据库

#### —— nmc_met_io程序库使用说明

国家气象中心天气预报技术研发室    
Dec., 2020    
Kan Dai

### 亚马逊开放和公用数据
------
* 亚马逊云计算面向科学研究需要, 支撑较为广泛的公开数据, [Registry of Open Data on AWS](https://registry.opendata.aws/)
* 对于地球或大气科学, 包含多种卫星遥感, 天气雷达, 再分析和数值模式预报资料

## 检索ERA5再分析数据
------
* ERA5是ECMWF第五代全球大气再分析数据.
* Intertrust Technologies Corporation公司利用[AWS公开数据库项目](https://aws.amazon.com/opendata/public-datasets/), 提供ERA5部分变量的云存储
  * 关于该数据库的说明, 详见https://github.com/planet-os/notebooks/blob/master/aws/era5-pds.md
  * 数据通过AWS的公共S3存储桶进行服务, 提供NetCDF和zarr两种格式
  * 可以使用 `aws s3 ls s3://era5-pds/ --no-sign-request` 命令来查询数据库中的内容
* 通用boto3可以访问读取数据文件信息及内容, 具体参考
  * [Accessing ERA5 Data on S3 Using Boto](https://github.com/planet-os/notebooks/blob/master/aws/era5-s3-via-boto.ipynb)
  * [Examining the Weather on Your Date of Birth](https://github.com/zflamig/birthday-weather)

In [1]:
# load necessary libraries
import numpy as np
import xarray as xr
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt

from nmc_met_io.retrieve_aws_server import retrieve_era5

In [2]:
# set time parameters
year = 1987
month = 12
datestart = '1987-12-02'
dateend = '1987-12-02 23:59'

# load data from AWS ERA5 S3 buckets
varname = 'air_pressure_at_mean_sea_level'
mslp = retrieve_era5(varname, year, month, datestart, dateend)

varname = 'air_temperature_at_2_metres'
airt = retrieve_era5(varname, year, month, datestart, dateend)

varname = 'precipitation_amount_1hour_Accumulation'
precip = retrieve_era5(varname, year, month, datestart, dateend)

In [3]:
%%time
# pre-process the data
precip_data = precip.precipitation_amount_1hour_Accumulation[12,:,:].values

mslp_data = mslp.air_pressure_at_mean_sea_level[12,:,:].values
mslp_data = mslp_data * 0.01 # Pa to hPa

airt_data = airt.air_temperature_at_2_metres[12,:,:].values
airt_data =  (airt_data - 273.15) * 9/5 + 32.0 # K to F

# get longitude and latitude coordinates
lat = mslp.lat.values
lon = mslp.lon.values
lons, lats = np.meshgrid(lon, lat)

ERROR:asyncio:Unclosed connection
client_connection: Connection<ConnectionKey(host='era5-pds.s3.amazonaws.com', port=443, is_ssl=True, ssl=None, proxy=None, proxy_auth=None, proxy_headers_hash=None)>
ERROR:asyncio:Unclosed connection
client_connection: Connection<ConnectionKey(host='era5-pds.s3.amazonaws.com', port=443, is_ssl=True, ssl=None, proxy=None, proxy_auth=None, proxy_headers_hash=None)>
ERROR:asyncio:Unclosed connection
client_connection: Connection<ConnectionKey(host='era5-pds.s3.amazonaws.com', port=443, is_ssl=True, ssl=None, proxy=None, proxy_auth=None, proxy_headers_hash=None)>


CPU times: user 29.6 s, sys: 4.04 s, total: 33.7 s
Wall time: 18min 40s


In [7]:
mslp.sel(time0='1987-12-31T22:00:00')

<xarray.Dataset>
Dimensions:                         (lat: 721, lon: 1440)
Coordinates:
  * lat                             (lat) float32 90.0 89.75 ... -89.75 -90.0
  * lon                             (lon) float32 0.0 0.25 0.5 ... 359.5 359.8
    time0                           datetime64[ns] 1987-12-31T22:00:00
Data variables:
    air_pressure_at_mean_sea_level  (lat, lon) float32 dask.array<chunksize=(150, 150), meta=np.ndarray>
Attributes:
    institution:  ECMWF
    source:       Reanalysis
    tilte:        ERA5 forecasts